In [7]:
!git clone https://github.com/RInkalshah93/ERA-V2-Assignment_Rinkal-Shah.git

Cloning into 'ERA-V2-Assignment_Rinkal-Shah'...
remote: Enumerating objects: 380, done.
remote: Counting objects: 100% (71/71), done.
remote: Compressing objects: 100% (46/46), done.
remote: Total 380 (delta 40), reused 56 (delta 25), pack-reused 309
Receiving objects: 100% (380/380), 35.40 MiB | 14.23 MiB/s, done.
Resolving deltas: 100% (157/157), done.


In [6]:
'''%cd ..
%cd ..
!rm -rf ERA-V2-Assignment_Rinkal-Shah'''

/
/


In [8]:
%cd ERA-V2-Assignment_Rinkal-Shah/S18_Assignment

/ERA-V2-Assignment_Rinkal-Shah/S18_Assignment


In [9]:
!pip install -r requirements.txt

In [10]:
from config_file import get_config, get_weights_file_path
from train import get_model, get_ds, run_validation

config = get_config()
config["batch_size"] = 16
config["preload"] = None
config["num_epochs"] = 1
import torch
torch.cuda.amp.autocast(enabled = True)


/usr/local/lib/python3.10/dist-packages/torchtext/datasets/__init__.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated and the last released version will be 0.18 (this one). You can silence this warning by calling the following at the beginnign of your scripts: `import torchtext; torchtext.disable_torchtext_deprecation_warning()`
  warnings.warn(torchtext._TORCHTEXT_DEPRECATION_MSG)
/usr/local/lib/python3.10/dist-packages/torchtext/data/__init__.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated and the last released version will be 0.18 (this one). You can silence this warning by calling the following at the beginnign of your scripts: `import torchtext; torchtext.disable_torchtext_deprecation_warning()`
  warnings.warn(torchtext._TORCHTEXT_DEPRECATION_MSG)


In [11]:
import warnings
from tqdm import tqdm
import os
from pathlib import Path

from torch.utils.tensorboard import SummaryWriter
import torch.nn as nn

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using Device:", device)

Path(config["model_folder"]).mkdir(parents = True, exist_ok = True)

train_dataloader, val_dataloader, tokenizer_src, tokenizer_tgt = get_ds(config)
model = get_model(config, tokenizer_src.get_vocab_size(), tokenizer_tgt.get_vocab_size()).to(device)

writer = SummaryWriter(config['experiment_name'])
loss_fn = nn.CrossEntropyLoss(ignore_index=tokenizer_src.token_to_id('[PAd]'), label_smoothing=0.1).to(device)

optimizer = torch.optim.Adam(model.parameters(),lr = config['lr'], eps=1e-9)


Using Device: cuda


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Max length of the source sentence : 309
Max length of the source target : 274


In [12]:
MAX_LR = 10**-3
STEPS_PER_EPOCH = len(train_dataloader)
EPOCHS = 18

In [13]:
scheduler = torch.optim.lr_scheduler.OneCycleLR(
                                                optimizer,
                                                max_lr =  MAX_LR,
                                                steps_per_epoch= STEPS_PER_EPOCH,
                                                epochs = EPOCHS,
                                                pct_start = int(0.3*EPOCHS)/EPOCHS if EPOCHS != 1 else 0.5,
                                                div_factor = 100,
                                                three_phase = False,
                                                final_div_factor = 100,
                                                anneal_strategy = "linear"

)

In [15]:
intial_epoch = 0
global_step =  0

scaler = torch.cuda.amp.GradScaler()
lr = [0.0]
for epoch in range(intial_epoch, 30):
    torch.cuda.empty_cache()
    model.train()
    batch_iterator = tqdm(train_dataloader, desc =f"Processing Epoch {epoch:02d}")
    for batch in batch_iterator:
        optimizer.zero_grad(set_to_none=True)
        encoder_input = batch['encoder_input'].to(device)
        decoder_input = batch['decoder_input'].to(device)
        encoder_mask = batch['encoder_mask'].to(device)
        decoder_mask = batch['decoder_mask'].to(device)

        with torch.autocast(device_type = 'cuda', dtype = torch.float16):
            encoder_output = model.encode(encoder_input, encoder_mask)
            decoder_output = model.decode(encoder_output, encoder_mask, decoder_input, decoder_mask)
            proj_output = model.project(decoder_output)

            label = batch['label'].to(device)

            loss = loss_fn(proj_output.view(-1, tokenizer_tgt.get_vocab_size()), label.view(-1))
        batch_iterator.set_postfix({"loss": f"{loss.item():6.3f}", "lr": f"{lr[-1]}"})

        writer.add_scalar('train loss', loss.item(), global_step)
        writer.flush()

        scaler.scal(loss).backward()

        scale = scaler.get_scale()
        scaler.step(optimizer)
        scaler.update()
        skip_lr_sched = (scale > scaler.get_scale())
        if not skip_lr_sched:
            scheduler.step()
        lr.append(scheduler.get_last_lr())

        global_step += 1

    run_validation(model, val_dataloader, tokenizer_src, tokenizer_tgt, config['seq_len'], device, lambda msg: batch_iterator.write(msg), global_step)

    model_filename = get_weights_file_path(config, f"{epoch:02d}")
    torch.save({
        'epoch' : epoch,
        'model_state_dict' : model.state_dict(),
        'optimizer_state_dict' : optimizer.state_dict(),
        'global_step' : global_step
        }, model_filename)

Processing Epoch 00:   0%|          | 0/1819 [00:00<?, ?it/s]


TypeError: cross_entropy_loss(): argument 'ignore_index' (position 5) must be int, not NoneType